In [1]:
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.yfinance import YFinanceTools
from phi.tools.duckduckgo import DuckDuckGo


In [2]:
from dotenv import load_dotenv
load_dotenv()


True

In [177]:
#Creating Agent-1
Research_agent_1=Agent(
    name= "Companies research Agent",
    role="Search the web for the information",
    model=Groq(id = "llama3-70b-8192"),
    tools=[DuckDuckGo()],
    instructions=["Give in detailed information","Always include sources"],
    show_tool_calls=True,
    markdown=True
)

In [181]:
#example working of Agent-1
company="Zomato"
response = Research_agent_1.run(company)

# Only extracts the 'content'
research_text = response.content

print(research_text)


Running:
 - duckduckgo_search(query=Zomato)


Running:
 - duckduckgo_news(query=Zomato news)

Here is the detailed information about Zomato:

**What is Zomato?**
Zomato is an online food delivery and restaurant discovery platform that allows users to search and discover restaurants, read reviews, and order food online. The company was founded in 2008 by Deepinder Goyal and Pankaj Chaddah.

**Recent News about Zomato**
Recently, there was a viral Reddit post that claimed Zomato was losing market share and forcing employees to order exclusively on Zomato. However, Zomato CEO Deepinder Goyal has rubbished these claims, calling them "utter nonsense". He reaffirmed that employees have the freedom to choose which food delivery platform they want to use.

**Sources:**
* [Business Today](https://www.msn.com/en-ph/money/topstories/utter-nonsense-zomato-ceo-deepinder-goyal-rubbishes-claims-on-market-share-loss-amid-viral-reddit-post/ar-AA1DEwSw)
* [Outlook Business](https://www.outlookbusiness.

In [182]:
#case genaration Agent
Case_genaration_Agent_2=Agent(
    name="Use case genarator Agent",
    role="Analyze the given industry/company research and generate AI/ML/GenAI based use cases to improve operations, customer experience, and efficiency.",
    model=Groq(id="llama3-70b-8192"),
    instructions=[
        "Take the input research carefully.",
        "Generate at least 5 detailed use cases.",
        "Ensure each use case mentions how GenAI, ML or LLMs can be applied.",
        "Suggest creative as well as industry-standard ideas."
    ],
    show_tool_calls=True,
    markdown=True
    
)

In [183]:
#example execution of Agent-2
use_case_response = Case_genaration_Agent_2.run(research_text)
use_case_response_text=use_case_response.content
print(use_case_response_text)

**Zomato: AI/ML/GenAI Based Use Cases to Improve Operations, Customer Experience, and Efficiency**

### Use Case 1: **Personalized Restaurant Recommendations**

**Implementation:** Utilize a Generative AI model, such as a transformers-based architecture, to analyze user behavior, search history, and review patterns to provide personalized restaurant recommendations.

**Benefits:**

* Enhanced customer experience through tailored suggestions
* Increased engagement and orders from recommended restaurants
* Improved user retention and loyalty

### Use Case 2: **Intelligent Order Routing and Fulfillment**

**Implementation:** Leverage Machine Learning algorithms, such as graph-based models, to optimize order routing and fulfillment. These algorithms can analyze various factors, including:

* Restaurant availability and capacity
* Delivery personnel location and availability
* Traffic patterns and road conditions
* Time of day and weather

**Benefits:**

* Reduced order fulfillment times
* 

In [184]:
#Convert Agent-2's verbose output into a search query.
"""Example Input: 
        "**Use Case 1**: AI-powered chatbot for Workday (leveraging NLP)..."
    Output: 
        "Workday chatbot NLP"
    """
import re

def extract_keywords(markdown_text: str, max_keywords=5) -> str:
    
    # Step 1: Remove markdown formatting
    clean_text = re.sub(r'\*+|`+|#+', '', markdown_text)
    
    # Step 2: Extract capitalized/noun phrases (crude but effective)
    keywords = re.findall(r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\b|\b\w{4,}\b', clean_text)
    
    # Step 3: Filter and limit keywords
    stopwords = {'for', 'and', 'the', 'with', 'using'} 
    keywords = [kw for kw in keywords if kw.lower() not in stopwords][:max_keywords]
    
    return " ".join(keywords)

#Example running:
#agent2_output = "**Use Case**: AI-powered Workday chatbot (NLP, LLMs)..."
print(extract_keywords(use_case_response_text))  # Output: "Workday chatbot NLP LLMs"

Zomato GenAI Based Use Cases Improve Operations Customer Experience


Using kaggle, hf, and github's api for data collection

In [185]:
from typing import List, Union
import json
# Ensure the output is a string (required for tool responses).

def format_output(results: Union[List[str], str]) -> str:
    if isinstance(results, list):
        return json.dumps(results)  # Convert list to JSON string
    return str(results)

In [186]:
from kaggle.api.kaggle_api_extended import KaggleApi
import os

def setup_kaggle():
    """Ensure kaggle.json exists or use environment variables."""
    if not os.path.exists(os.path.expanduser("~/.kaggle/kaggle.json")):
        raise FileNotFoundError("Place kaggle.json in ~/.kaggle/ or set KAGGLE_USERNAME/KAGGLE_KEY env vars.")

# Kaggle
def search_kaggle_datasets(query: str, max_results: int = 3) -> str:  # Return string!
    try:
        api = KaggleApi()
        api.authenticate()
        datasets = api.dataset_list(search=query)[:max_results]
        links = [f"https://www.kaggle.com/datasets/{d.ref}" for d in datasets]
        return format_output(links)  # Ensure string output
    except Exception as e:
        return format_output(f"Kaggle Error: {e}")

In [187]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()  # No errors = success!

In [188]:
from huggingface_hub import list_datasets

def search_hf_datasets(query: str, max_results: int = 3) -> str:
    try:
        datasets = list(list_datasets(search=query))[:max_results]
        links = [f"https://huggingface.co/datasets/{d.id}" for d in datasets]
        return format_output(links)
    except Exception as e:
        return format_output(f"HF Error: {e}")

In [189]:
import requests
from bs4 import BeautifulSoup

def search_github_datasets(query: str, max_results: int = 3) -> str:
    try:
        # ... (your scraping code)
        links = [...]  # List of GitHub links
        return format_output(links)
    except Exception as e:
        return format_output(f"GitHub Error: {e}")

In [190]:
Dataset_Collector_Agent_3 = Agent(
    name="Dataset Collector Agent",
    model=Groq(id="llama3-70b-8192"),
    tools=[search_kaggle_datasets, search_hf_datasets, search_github_datasets],
    instructions=[
        "Prioritize Kaggle > HuggingFace > GitHub for datasets.",
        "Format: [Platform: Title](URL)",
        "Skip if no results found.",
        "Never hallucinate links!"
    ],
    markdown=True
)


In [ ]:
import re

def extract_use_case_titles(markdown_text: str) -> list[str]:
    """Extract ### headings from markdown (use case titles)"""
    return re.findall(r'### \d+\. \*\*(.*?)\*\*', markdown_text)

# Example:
agent2_output = """
## Use Cases
### 1. **Intelligent Inventory Management**
### 2. **AI-Powered Chatbots**
"""
titles = extract_use_case_titles(use_case_response_text) 

[]


In [196]:
def create_search_query(title: str) -> str:
    """Convert use case title to dataset search query"""
    # Keep only nouns/verbs (remove connectors)
    keywords = re.sub(r'\b(for|using|with|the)\b', '', title)
    return f"{keywords} dataset"

# Example:
create_search_query("Intelligent Inventory Management") 
# -> "Intelligent Inventory Management dataset"

'Intelligent Inventory Management dataset'

In [197]:
def find_datasets_for_use_cases(use_case_titles: list[str]) -> dict:
    results = {}
    for title in use_case_titles:
        query = create_search_query(title)
        response = Dataset_Collector_Agent_3.run(query)
        results[title] = response.content
    return results

# Usage:
datasets_by_use_case = find_datasets_for_use_cases(titles)

In [198]:
def generate_ai_use_cases_with_datasets(company: str) -> str:
    # Step 1: Run Agent 1 → Agent 2
    research = Research_agent_1.run(f"Find details about {company}").content
    use_cases_markdown = Case_genaration_Agent_2.run(research).content
    
    # Step 2: Process Agent 2's output
    use_case_titles = extract_use_case_titles(use_cases_markdown)
    datasets_by_use_case = find_datasets_for_use_cases(use_case_titles)
    
    # Step 3: Generate structured output
    output = ["## AI Use Cases with Recommended Datasets"]
    for title in use_case_titles:
        output.append(f"### {title}")
        output.append(datasets_by_use_case.get(title, "No relevant datasets found"))
    
    return "\n".join(output)

In [199]:
# Test with Walmart
result = generate_ai_use_cases_with_datasets(company)
print(result)

## AI Use Cases with Recommended Datasets
### Personalized Restaurant Recommendations
[Kaggle: Restaurant Recommendations](https://www.kaggle.com/datasets/samayashar/restaurant-recommendations)
[Kaggle: Dubai's Finest Eats - 13K Restaurant Dataset](https://www.kaggle.com/datasets/shahriarrahman009/dubais-finest-eats-13k-restaurant-dataset)
[Kaggle: Global Zomato Dataset](https://www.kaggle.com/datasets/harishkumardatalab/global-zomato-dataset)
### Real-time Menu Item Suggestion
It seems like the tools didn't yield any results. I'll respond directly without using a tool. 

No relevant datasets were found for "Real-time Menu Item Suggestion dataset".
### Predictive Food Preparation and Delivery Times
[Kaggle: Food Delivery Order History Data](https://www.kaggle.com/datasets/sujalsuthar/food-delivery-order-history-data)
[Kaggle: Food Delivery Time](https://www.kaggle.com/datasets/willianoliveiragibin/food-delivery-time)
### Restaurant Performance Analytics
[Kaggle: Zomato Delivery Operati